In [ ]:
import pandas as pd
df = pd.read_csv("Data/18_Customer_data.csv", encoding="cp949")

In [30]:
import pandas as pd
df = pd.read_excel("Data/customer.cleand.xlsx")
df

,고객ID,성별,연령,결혼여부,부양자유무,부양자수,추천횟수,영수증발급여부,과금방식,보안서비스,...,고객이탈여부,이탈가능점수,이탈유형,이탈사유,LTV,만족도,데이터사용료,로밍사용료,총로밍요금,과금액
0,C-14721,여성,43,Yes,No,0,1,Yes,신용카드,Yes,...,No,41,NaN,NaN,4743,3,0,28140,2025070,11675100
1,C-15361,여성,33,Yes,Yes,3,1,Yes,계좌이체,Yes,...,No,49,NaN,NaN,5475,4,0,46440,3347170,11641300
2,C-11025,남성,75,Yes,No,0,5,Yes,계좌이체,Yes,...,No,64,NaN,NaN,4404,3,135600,24180,1738620,11628400
3,C-12687,여성,28,Yes,Yes,1,10,Yes,계좌이체,Yes,...,No,54,NaN,NaN,6173,3,0,53110,3825500,11623300
4,C-16549,여성,26,Yes,No,0,4,Yes,계좌이체,Yes,...,No,44,NaN,NaN,4697,3,0,27460,1978630,11603300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6623,C-13915,남성,35,No,Yes,2,0,No,신용카드,No,...,Yes,80,Attitude,Attitude of service provi,5437,3,0,14460,14460,43600
6624,C-15667,남성,52,No,No,0,0,Yes,신용카드,No,...,Yes,95,Competitor,Product dissatisfaction,3448,1,0,16050,16050,43100
6625,C-14085,남성,52,No,No,0,0,No,이체/메일확인,No,...,Yes,66,Dissatisfaction,Service dissatisfaction,4152,2,0,3500,3500,42700
6626,C-11991,여성,64,No,No,0,0,Yes,신용카드,No,...,Yes,85,Attitude,Network reliability,2783,2,0,12090,12090,42400


In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# 데이터프레임 예시 (df는 이미 존재하는 데이터프레임으로 가정)
# 종속변수: 이탈여부 (0 또는 1)
# 독립변수: 이탈가능점수, 데이터사용량, 데이터사용료, LTV, 총로밍요금, 만족도

# 독립변수(X)와 종속변수(y) 지정
X = df[['이탈가능점수', '데이터사용량', '데이터사용료', 'LTV', '총로밍요금', '만족도']]
y = df['고객이탈여부']

# 데이터 분할: 학습 데이터와 테스트 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 생성
log_reg = LogisticRegression(max_iter=1000)

# 모델 학습
log_reg.fit(X_train, y_train)

# 예측 결과
y_pred = log_reg.predict(X_test)

# 모델 평가
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 회귀 계수 출력
print("\n회귀 계수:")
coefficients = pd.DataFrame({
    '변수': X.columns,
    '회귀 계수': log_reg.coef_[0]
})
print(coefficients)


Confusion Matrix:
[[1538   18]
 [  37  396]]

Classification Report:
              precision    recall  f1-score   support

          No       0.98      0.99      0.98      1556
         Yes       0.96      0.91      0.94       433

    accuracy                           0.97      1989
   macro avg       0.97      0.95      0.96      1989
weighted avg       0.97      0.97      0.97      1989


회귀 계수:
       변수         회귀 계수
0  이탈가능점수  1.896376e-01
1  데이터사용량  1.285244e-03
2  데이터사용료 -1.118539e-06
3     LTV -1.245193e-04
4   총로밍요금 -7.129194e-07
5     만족도 -4.511096e+00


3/11

In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# ====== [1] 데이터 로드 및 전처리 ======

# 1-1) "Yes"/"No" 값을 1과 0으로 변환
print("\n✅ Step 1: 고객이탈여부 변환")
df['고객이탈여부'] = df['고객이탈여부'].map({'Yes': 1, 'No': 0})
print(df['고객이탈여부'].value_counts())  # 변환 확인

# 1-2) 데이터 타입 확인 및 변환 (숫자로 변환)
print("\n✅ Step 2: 데이터 타입 변환")
for col in df.columns:
    df[col] = df[col].astype(str)  # 문자열 변환
    df[col] = df[col].str.replace(r'[^0-9.-]', '', regex=True)  # 숫자, 소수점, 음수 기호 제외한 문자 제거
    df[col] = pd.to_numeric(df[col], errors='coerce')  # 숫자로 변환 (변환 불가 값은 NaN 처리)

# 1-3) 결측값(NaN)이 있으면 평균값으로 대체
df = df.fillna(df.mean())

# 1-4) 데이터 확인 (고객이탈여부 값 확인)
print("\n✅ Step 3: 고객이탈여부 값 분포 확인")
print(df['고객이탈여부'].value_counts())

# 1-5) 독립변수(X)와 종속변수(y) 분리
X = df[['이탈가능점수', '데이터사용량', '데이터사용료', 'LTV', '총로밍요금', '만족도']]
y = df['고객이탈여부']

# 1-6) y 값이 0 또는 1만 남도록 필터링 (혹시 이상한 값이 남았을 경우)
df = df[df['고객이탈여부'].isin([0, 1])]
y = df['고객이탈여부']
X = X.loc[y.index]  # 인덱스 맞추기

print("\n✅ Step 4: 필터링 후 고객이탈여부 값 분포 확인")
print(y.value_counts())

if y.empty or X.empty:
    raise ValueError("❌ 데이터가 없습니다. 필터링 과정에서 모든 데이터가 삭제되었을 가능성이 있습니다.")

# ====== [2] 로지스틱 회귀 분석 (statsmodels) - 회귀 계수와 p-value 확인 ======

# 2-1) 상수항 추가 (절편 포함)
X_const = sm.add_constant(X)

# 2-2) 로지스틱 회귀 모델 적합
print("\n✅ Step 5: 로지스틱 회귀 모델 적합")
log_reg_sm = sm.Logit(y, X_const)
result = log_reg_sm.fit()

# 2-3) 결과 출력 (회귀 계수, p-value 등 확인)
print("\n===== Statsmodels 로지스틱 회귀 결과 =====")
print(result.summary())

# ====== [3] 로지스틱 회귀 분석 (scikit-learn) - 모델 평가 ======

# 3-1) 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3-2) scikit-learn 로지스틱 회귀 모델 생성 및 학습
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# 3-3) 예측 수행
y_pred = log_reg.predict(X_test)

# 3-4) 모델 평가 - 혼동 행렬 및 분류 보고서 출력
print("\n===== Confusion Matrix =====")
print(confusion_matrix(y_test, y_pred))

print("\n===== Classification Report =====")
print(classification_report(y_test, y_pred))

# 3-5) 회귀 계수 출력
coefficients = pd.DataFrame({
    '변수': X.columns,
    '회귀 계수': log_reg.coef_[0]
})
print("\n===== 회귀 계수 =====")
print(coefficients)



✅ Step 1: 고객이탈여부 변환
고객이탈여부
0    5161
1    1467
Name: count, dtype: int64

✅ Step 2: 데이터 타입 변환

✅ Step 3: 고객이탈여부 값 분포 확인
고객이탈여부
0    5161
1    1467
Name: count, dtype: int64

✅ Step 4: 필터링 후 고객이탈여부 값 분포 확인
고객이탈여부
0    5161
1    1467
Name: count, dtype: int64

✅ Step 5: 로지스틱 회귀 모델 적합
         Current function value: 0.069945
         Iterations: 35

===== Statsmodels 로지스틱 회귀 결과 =====
                           Logit Regression Results                           
Dep. Variable:                 고객이탈여부   No. Observations:                 6628
Model:                          Logit   Df Residuals:                     6621
Method:                           MLE   Df Model:                            6
Date:                Tue, 11 Mar 2025   Pseudo R-squ.:                  0.8677
Time:                        13:08:09   Log-Likelihood:                -463.59
converged:                      False   LL-Null:                       -3503.5
Covariance Type:            nonrobust   LLR p-value:         

C:\Users\campus4D019\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터프레임 예시 (df는 이미 존재하는 데이터프레임으로 가정)
# 종속변수: 고객이탈여부 (0 또는 1)
# 독립변수: 이탈가능점수, 데이터사용량, 데이터사용료, LTV, 총로밍요금, 만족도

# Step 1: 데이터 전처리
# 결측값 처리
df = df.fillna(df.mean())  # 결측값을 평균값으로 채움

# Step 2: 독립변수(X)와 종속변수(y) 지정
X = df[['이탈가능점수', '데이터사용량', '데이터사용료', 'LTV', '총로밍요금', '만족도']]
y = df['고객이탈여부']

# Step 3: 데이터 분할 (학습 데이터와 테스트 데이터로 나누기)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Feature scaling (스케일링 적용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 학습 데이터 스케일링
X_test_scaled = scaler.transform(X_test)  # 테스트 데이터 스케일링

# Step 5: 상수항 추가
X_train_scaled_const = sm.add_constant(X_train_scaled)  # 학습 데이터에 상수항 추가
X_test_scaled_const = sm.add_constant(X_test_scaled)  # 테스트 데이터에 상수항 추가

# Step 6: 로지스틱 회귀 모델 적합
log_reg_sm = sm.Logit(y_train, X_train_scaled_const)
result = log_reg_sm.fit(maxiter=2000)  # max_iter를 2000으로 설정하여 수렴 유도

# Step 7: 결과 출력
print("===== Statsmodels 로지스틱 회귀 결과 =====")
print(result.summary())

# Step 8: 모델 예측 및 평가
y_pred = result.predict(X_test_scaled_const)
y_pred_class = np.round(y_pred)  # 예측값을 0 또는 1로 반올림

from sklearn.metrics import confusion_matrix, classification_report
print("\n===== Confusion Matrix =====")
print(confusion_matrix(y_test, y_pred_class))

print("\n===== Classification Report =====")
print(classification_report(y_test, y_pred_class))



TypeError: unsupported operand type(s) for +: 'int' and 'str'